In [ ]:
from IPython.core.display import Image, display
display(Image(url='https://www.greekmythology.com/images/mythology/aegis_228.jpg', width=200, unconfined=True))

AEGIS: Astrophysical Event Generator for Integration with Simulation-based inference

Make sure you're working with a kernal that has healpy, astropy, and sbi installed.

Let's get started generating some photon maps with an AEGIS object. First, let's import some needed modules.

In [ ]:
import aegis
import numpy as np
import healpy as hp
import pickle as pk
from astropy import units as u
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
import sys
from sbi.inference import SNLE, SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from getdist import plots, MCSamples

%matplotlib inline

Now, let's initialize an AEGIS. First, we must define a few parameters that control how the AEGIS will generate sources and photons.

In [ ]:
parameter_range = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [2000, 100000] #MeV
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
luminosity_range = 10.0**np.array([30, 37])
max_radius = 8.5 + 20*2 #kpc
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [ ]:
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen, my_AEGIS.Emin_gen, my_AEGIS.Emax_gen = angular_cut_gen, lat_cut_gen, energy_range_gen[0], energy_range_gen[1]

Our first source will be a simple member of the independent_spherical_single_spectrum class. First, its als is defined. Then, the als and source class are added to the AEGIS.

In [ ]:
def R_s(r, params):
    A = params[0]
    r_s = 1 #kpc
    return A * np.exp(-r/r_s)

def Theta_s(theta, params):
    return np.ones(np.shape(theta))

def Phi_s(phi, params):
    return np.ones(np.shape(phi))

def L(L, params):
    L_b = 1.56e37/2.35 #photons/s
    return np.exp(-1000*L/L_b)

def spec(energy, params):
    return np.ones(np.size(energy))

als_s = [(R_s, Theta_s, Phi_s), L, spec]

my_AEGIS.abun_lum_spec = [als_s]
my_AEGIS.source_class_list = ['independent_spherical_single_spectrum']

With a source now in our AEGIS, we can generate a realization of these sources, and the photons we detect from them. This is done by calling the AEGIS methods create_sources and generate_photons_from_sources, respectively. An optional argument in these methods is the grains parameter. It controls the number of bins in all arrays used to generate the data, i.e. the granularity.


In [ ]:
input_params = [10000]

source_info = my_AEGIS.create_sources(input_params, grains = 10000)
photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info)

N_side = 2**6
heatmap = np.histogram(hp.ang2pix(N_side, source_info['angles'][:,0], source_info['angles'][:,1]), bins = 12*N_side**2, range = [0, 12*N_side**2])
hp.mollview(heatmap[0], title = 'Point Sources')

heatmap = np.histogram(hp.ang2pix(N_side, photon_info['angles'][:,0], photon_info['angles'][:,1]), bins = 12*N_side**2, range = [0, 12*N_side**2])
hp.mollview(heatmap[0], title = 'Photons')

fig, ax = plt.subplots()
ax.set_title('Photon Energies')
ax.set_yscale('log')
#ax.set_xscale('log')
ax.set_ylabel('Number of Photons')
ax.set_xlabel('Energy [MeV]')
ax.hist(photon_info['energies'], bins = 30, range = energy_range_gen)

print('Number of sources: ', np.size(source_info['distances']))
print('Number of photons: ', np.size(photon_info['energies']))
print('Average photon energy: ', np.mean(photon_info['energies'])/1000, ' GeV')
print('Average luminosity/source: ', np.mean(source_info['luminosities'])*np.mean(photon_info['energies'])*u.MeV.to('erg'), r'$ erg s^{-1}$')
print('Approximate Flux: ', np.mean(photon_info['energies'])*u.MeV.to('erg')*np.size(photon_info['energies'])/(exposure*u.yr.to('s')), r'$ erg cm^{-2}s^{-1}$')
print('Size of pixels (deg):', hp.nside2resol(N_side)*u.rad.to('deg'))

Now, let's replace our previous source with a simple independent_cylindrical_single_spectrum source and regenerate the data

In [ ]:
def R_c(r, params):
    A = params[0]
    r_s = 3 #kpc
    return A * np.exp(-r/r_s)

def Z_c(z, params):
    z_s = 3 #kpc
    return np.exp(-np.abs(z)/z_s)

def Phi_c(phi, params):
    return np.ones(np.shape(phi))

def L(L, params):
    L_b = 1.56e37/2.35 #photons/s
    return np.exp(-1000*L/L_b)

def spec(energy, params):
    return np.ones(np.size(energy))

als_c = [(R_c, Z_c, Phi_c), L, spec]
my_AEGIS.abun_lum_spec = [als_c]
my_AEGIS.source_class_list = ['independent_cylindrical_single_spectrum']

In [ ]:
input_params = [10000]

source_info = my_AEGIS.create_sources(input_params, grains = 10000)
photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info)

N_side = 2**6
heatmap = np.histogram(hp.ang2pix(N_side, source_info['angles'][:,0], source_info['angles'][:,1]), bins = 12*N_side**2, range = [0, 12*N_side**2])
hp.mollview(heatmap[0], title = 'Point Sources')

heatmap = np.histogram(hp.ang2pix(N_side, photon_info['angles'][:,0], photon_info['angles'][:,1]), bins = 12*N_side**2, range = [0, 12*N_side**2])
hp.mollview(heatmap[0], title = 'Photons')

fig, ax = plt.subplots()
ax.set_title('Photon Energies')
ax.set_yscale('log')
#ax.set_xscale('log')
ax.set_ylabel('Number of Photons')
ax.set_xlabel('Energy [MeV]')
ax.hist(photon_info['energies'], bins = 30, range = energy_range_gen)

print('Number of sources: ', np.size(source_info['distances']))
print('Number of photons: ', np.size(photon_info['energies']))
print('Average photon energy: ', np.mean(photon_info['energies'])/1000, ' GeV')
print('Average luminosity/source: ', np.mean(source_info['luminosities'])*np.mean(photon_info['energies'])*u.MeV.to('erg'), r'$ erg s^{-1}$')
print('Approximate Flux: ', np.mean(photon_info['energies'])*u.MeV.to('erg')*np.size(photon_info['energies'])/(exposure*u.yr.to('s')), r'$ erg cm^{-2}s^{-1}$')
print('Size of pixels (deg):', hp.nside2resol(N_side)*u.rad.to('deg'))